## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-07-34-39 +0000,bbc,Plans to cut cash Isa tax-free allowance on hold,https://www.bbc.com/news/articles/cqjq9yxkkrvo
1,2025-07-11-07-25-27 +0000,bbc,UK economy shrank unexpectedly in May,https://www.bbc.com/news/articles/cq6mvem8neno
2,2025-07-11-07-16-21 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
3,2025-07-11-07-00-59 +0000,bbc,Kurdish PKK to lay down arms in big step towar...,https://www.bbc.com/news/articles/cyvj5v67ly9o
4,2025-07-11-06-59-00 +0000,wsj,U.K. Economy Unexpectedly Slumps to Second Str...,https://www.wsj.com/economy/global/u-k-economy...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,12
8,uk,6
55,tariffs,5
239,brazil,4
284,six,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
48,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,36
64,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,33
67,2025-07-10-11-30-07 +0000,bbc,Brazil vows to match US tariffs after Trump th...,https://www.bbc.com/news/articles/c62dvyv60z7o,26
34,2025-07-10-20-10-28 +0000,nyt,"Targeting Brazil, Trump Tests Legal Limit of H...",https://www.nytimes.com/2025/07/10/us/politics...,25
14,2025-07-11-00-52-29 +0000,latimes,New Trump rule immediately bans undocumented i...,https://www.latimes.com/california/story/2025-...,25


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
48,36,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
2,22,2025-07-11-07-16-21 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
75,17,2025-07-10-10-00-00 +0000,latimes,L.A. has never experienced loss on this scale....,https://www.latimes.com/california/story/2025-...
37,16,2025-07-10-19-45-28 +0000,bbc,Watch: Rare shelf cloud seen over Maryland ami...,https://www.bbc.com/news/videos/cwyg93lzqd7o
17,15,2025-07-10-23-44-36 +0000,latimes,Los Angeles County sheriff's deputy pleads gui...,https://www.latimes.com/california/story/2025-...
14,13,2025-07-11-00-52-29 +0000,latimes,New Trump rule immediately bans undocumented i...,https://www.latimes.com/california/story/2025-...
23,13,2025-07-10-22-45-11 +0000,latimes,"As L.A. bakes, duration of heat waves is accel...",https://www.latimes.com/california/story/2025-...
26,12,2025-07-10-21-57-54 +0000,latimes,"Cal State L.A. allows online classes, excused ...",https://www.latimes.com/california/story/2025-...
84,12,2025-07-10-08-22-55 +0000,bbc,'A huge explosion-like sound' - Survivors reca...,https://www.bbc.com/news/articles/cjel1g15qxpo
24,11,2025-07-10-22-13-02 +0000,latimes,Officials denounce immigration guards camped a...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
